In [ ]:
import scanpy as sc
import numpy as np
import os
import anndata2ri
import pathlib
from scipy import io
import anndata#
import pandas as pd
from tqdm import tqdm
import argparse
import sys
import re
import scvelo as scv

# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
# anndata2ri.activate()

#Loading the rpy2 extension enables cell magic to be used
#This runs R code in jupyter notebook cells
# %load_ext rpy2.ipython

sc.settings.verbosity = 3
# sc.logging.print_versions()

import warnings
warnings.filterwarnings("ignore")

outdir = "/media/hieunguyen/HD01/outdir/CRC1382/SBharadwaj_20250102"

path_to_main_src = "/home/hieunguyen/CRC1382/src_2023/SBharadwaj/release"
samplesheet = pd.read_csv(os.path.join(path_to_main_src, "SampleSheet_all_seurat_objects.csv"))
samplesheet["dataset_name"] = samplesheet.apply(
    lambda row: f"{row['dataset_name']}_reIntegration" if row["reIntegration"] == "yes" else row["dataset_name"],
    axis=1
)
samplesheet = samplesheet.fillna("")

#####------------------------------------------------------------#####
##### READ LOOM DATA
#####------------------------------------------------------------#####
path_to_loom_data = f"{outdir}/loom"
all_looms = [item for item in pathlib.Path(path_to_loom_data).glob("*.loom")]

velodata_dict = dict()
for input_loom in tqdm(all_looms):
    velodata = scv.read_loom(input_loom)
    samplename = input_loom.name.replace(".loom", "")

    new_obs = ["{}_{}_{}-1".format(samplename, samplename, item.split(":")[1].replace("x", "")) for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata.obs["barcode"] = new_obs
    velodata.var_names_make_unique()
    pattern = re.compile('_[A|T|G|C]*-')
    new_obs = [samplename + pattern.search(string = item).group() + item.split("-")[1]
            for item in velodata.obs.index]
    velodata.obs.index = new_obs
    velodata_dict[samplename] = velodata
    velodata.var_names_make_unique()

all_velodata = velodata_dict[list(velodata_dict.keys())[0]]
for data in list(velodata_dict.keys())[1:]:
    all_velodata = all_velodata.concatenate(velodata_dict[data])
    
new_obs = [item.split("-")[0] + "-1" for item in all_velodata.obs.index]
all_velodata.obs.index = new_obs
all_velodata.var_names_make_unique()

#####------------------------------------------------------------#####
##### MAIN FUNCTION
#####------------------------------------------------------------#####
for index, row in samplesheet.iterrows():
    PROJECT = row['PROJECT']
    dataset_name = row['dataset_name']
    path_to_s_obj = row['path']
    re_integration = row['reIntegration']
    
    re_integration = row['reIntegration']
    if dataset_name == "full":
        if re_integration in ["yes", ""]:
            to_run_clusters = ["cca.cluster.0.5", "cell.annotation"]
            reduction_name = "cca_UMAP"
        else:
            to_run_clusters = ["seurat_clusters", "cell.annotation"]
            reduction_name = "SCT_UMAP"
    else:
        if re_integration in ["yes", ""]:
            to_run_clusters = ["cca.cluster.0.5"]
            reduction_name = "cca_UMAP"
        else:
            to_run_clusters = ["seurat_clusters"]
            reduction_name = "SCT_UMAP"
    
    path_to_s_obj = path_to_s_obj.replace(".rds", ".addedInfo.rds")
    
    path_to_main_output = os.path.join(outdir, PROJECT, "data_analysis")
    path_to_08_output = os.path.join(path_to_main_output, "08_output", dataset_name)
    os.makedirs(path_to_08_output, exist_ok=True)
    
    path_to_seurat2anndata = os.path.join(path_to_08_output, "seurat2anndata")
    os.makedirs(path_to_seurat2anndata, exist_ok=True)

    if PROJECT == "SBharadwaj_20240318_Sample_3_6":
        to_run_clusters = [cluster for cluster in to_run_clusters if cluster != "cell.annotation"]
    
    for cluster_name in to_run_clusters:
        object_name = f"{PROJECT}_{dataset_name}_{cluster_name}"

        adata = sc.read_h5ad(os.path.join(path_to_seurat2anndata, '{}.h5ad'.format(object_name)))
        obsdf = adata.obs.copy()
        obsdf["barcode"] = obsdf[["barcode", "name"]].apply(lambda x: x[0].replace(f"{x[1]}_{x[1]}", f"{x[1]}"), axis = 1) 
        adata.obs.index = obsdf.barcode.values
        adata.var_names_make_unique()

        colordf = pd.read_csv(os.path.join(path_to_seurat2anndata, 'colordf_{}.csv'.format(object_name)))
        colors = colordf["color"].values
        
        #####------------------------------------------------------------#####
        ##### Data pre-processing
        #####------------------------------------------------------------#####
        merge_data = scv.utils.merge(adata, all_velodata)
        assert merge_data.obs.shape[0] == adata.obs.shape[0]
        #####------------------------------------------------------------#####
        ##### Merge data and preprocessing again
        #####------------------------------------------------------------#####
        scv.pp.filter_genes(merge_data, min_shared_counts=20)
        scv.pp.normalize_per_cell(merge_data)
        scv.pp.filter_genes_dispersion(merge_data, n_top_genes=2000)
        scv.pp.log1p(merge_data)

        scv.pp.filter_and_normalize(merge_data, min_shared_counts=20, n_top_genes=2000)
        scv.pp.moments(merge_data, n_pcs=30, n_neighbors=30)

        #####------------------------------------------------------------#####
        ##### RNA velocity inference
        #####------------------------------------------------------------#####
        scv.tl.velocity(merge_data)
        scv.tl.velocity_graph(merge_data)

        #####------------------------------------------------------------#####
        ##### save streamline RNA velocity
        #####------------------------------------------------------------#####
        scv.pl.velocity_embedding_stream(merge_data, 
                                        dpi=120, 
                                        arrow_size=2, basis = "X_umap",
                                        color="cca.cluster.0.5",
                                figsize = (12, 12), 
                                fontsize=20, 
                                legend_fontsize = 30, 
                                frameon=True, 
                                save="streamline_{}.svg".format(object_name),
                                palette = colors)

        ##### Gene ranking
        scv.tl.rank_velocity_genes(merge_data, groupby=cluster_name, min_corr=.3)
        df_gene_ranking = scv.DataFrame(merge_data.uns['rank_velocity_genes']['names'])
        df_gene_ranking.to_excel(f"./figures/{object_name}_gene_ranking.xlsx")

        ##### Pseudotime plot
        scv.tl.velocity_pseudotime(merge_data)
        scv.pl.scatter(merge_data, 
                    color='velocity_pseudotime', 
                    cmap='gnuplot', 
                    basis = "X_umap",
                    figsize = (15, 15), 
                    fontsize=20, 
                    legend_fontsize = 30, 
                    frameon=True, 
                    save="pseudotime_{}.svg".format(object_name),
                    palette = colors)

        ##### Speed and coherence
        scv.tl.velocity_confidence(merge_data)
        keys = 'velocity_length', 'velocity_confidence'
        scv.pl.scatter(merge_data, c=keys, cmap='coolwarm', perc=[5, 95], basis = "X_umap", frameon=True, 
                    figsize = (10, 10), save="speed_coherence_{}.svg".format(object_name))

        df = merge_data.obs.groupby(cluster_name)[keys].mean().T
        df.to_excel(f"./figures/{object_name}_speed_coherence.xlsx")